In [1]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data= data.drop(['RowNumber','CustomerId','Surname'], axis =1)

In [5]:
# Encode categorical variables
encode_gender= LabelEncoder()
data['Gender']= encode_gender.fit_transform(data['Gender'])

In [6]:
# encode geography
encode_geo= OneHotEncoder(handle_unknown='ignore')
geo_encoded=encode_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded, columns=encode_geo.get_feature_names_out(['Geography']))

In [8]:
data=pd.concat([data.drop('Geography', axis=1),geo_encoded_df], axis=1)

In [11]:
#Split into independent and dependent features
X= data.drop('EstimatedSalary', axis=1)
Y= data['EstimatedSalary']


In [13]:
#Split the data for training and testing
X_train, X_test, Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [14]:
#Scaling the numerical features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [15]:
# Save encoders and scaler for later use
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(encode_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(encode_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# ANN Regression 

In [16]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [19]:
model= Sequential([
    Dense(64,activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) # Output Layer for Regression (activation function default: linear)
])

In [21]:
#Compile model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Setup Tensorboard for training
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [23]:
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
# Setup earlystopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [25]:
# Train the model
history= model.fit(
    X_train, Y_train,
    validation_data=(X_test,Y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 4ms/step - loss: 100367.6328 - mae: 100367.6328 - val_loss: 98477.4844 - val_mae: 98477.4844
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99477.0078 - mae: 99477.0078 - val_loss: 96685.1250 - val_mae: 96685.1250
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 96350.4766 - mae: 96350.4766 - val_loss: 92074.9766 - val_mae: 92074.9766
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 90174.2188 - mae: 90174.2188 - val_loss: 84441.2422 - val_mae: 84441.2422
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 81303.8984 - mae: 81303.8984 - val_loss: 74868.6016 - val_mae: 74868.6016
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 71321.7109 - mae: 71321.7109 - val_loss: 65513.8008 - val_mae: 65513.8008
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 62393.9531 - mae: 62393.953

In [26]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir regressionlogs/fit 

Reusing TensorBoard on port 6006 (pid 30028), started 0:01:22 ago. (Use '!kill 30028' to kill it.)

In [30]:
## Evaluate model on test data
test_loss, test_mae= model.evaluate(X_test,Y_test)
print(f'Test MAE:{test_mae}')

63/63 [==============================] - 0s 2ms/step - loss: 50275.9844 - mae: 50275.9844
Test MAE:50275.984375


In [31]:
model.save('regression_model.h5')

c:\Users\iamsh\Python\envs\denv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
